In [1]:
import numpy as np
import pandas as pd

In [2]:
import kagglehub

path = kagglehub.dataset_download("oddrationale/mnist-in-csv")

print("Path to dataset files:", path)

C:\Users\vgrig\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\vgrig\.cache\kagglehub\datasets\oddrationale\mnist-in-csv\versions\2


In [3]:
train_csv = pd.read_csv(path + "/mnist_train.csv")
test_csv = pd.read_csv(path + "/mnist_test.csv")
# этот файл загружается отдельно
train_2_csv = pd.read_csv("./data/train.csv")
train_csv

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59997,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
x_train = np.array(train_csv.iloc[:, 1:].values)
x_2_train = np.array(train_2_csv.iloc[:, 1:].values)
y_train = np.array(train_csv.iloc[:, 0].values, )
y_2_train = np.array(train_2_csv.iloc[:, 0].values)

x_train = np.array(np.concatenate([x_train.data, x_2_train.data], axis=0))
y_train = np.array(np.concatenate([y_train.data, y_2_train.data], axis=0))

print(x_train.shape)

x_train = x_train / 255.0

x_test = np.array(test_csv.iloc[:, 1:].values)
y_test = np.array(test_csv.iloc[:, 0].values)

x_test = x_test / 255.0

print(x_test.shape)

(102000, 784)
(10000, 784)


In [5]:
def to_one_hot(y, num_classes):
    return np.eye(num_classes)[y]


y_train = to_one_hot(y_train, 10)
y_test = to_one_hot(y_test, 10)

In [6]:
def tanh(x):
    return np.tanh(x)


def tanh2deriv(out):
    return 1 - (out * out)


def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

In [7]:
alpha, iterations, hidden_size = (0.1, 150, 256)
pixels, num_labels = (784, 10)
batch_size = 256

In [8]:
np.random.seed(1)
weight_0_1 = 0.02 * np.random.random((pixels, hidden_size)) - 0.01
weight_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1
print(weight_0_1.shape)
print(weight_1_2.shape)

(784, 256)
(256, 10)


In [9]:
for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(x_train) / batch_size)):
        batch_start, batch_end = ((i * batch_size), (i * batch_size + batch_size))
        layer_0 = x_train[batch_start:batch_end]
        layer_1 = tanh(np.dot(layer_0, weight_0_1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1, weight_1_2))
        for k in range(batch_size):
            correct_cnt += int(np.argmax(layer_2[k:k + 1]) == np.argmax(y_train[batch_start + k:batch_start + k + 1]))
        layer_2_delta = (y_train[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weight_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask

        weight_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        weight_0_1 += alpha * layer_0.T.dot(layer_1_delta)

    test_correct_cnt = 0
    for i in range(len(x_test)):
        layer_0 = x_test[i:i + 1]
        layer_1 = tanh(np.dot(layer_0, weight_0_1))
        layer_2 = np.dot(layer_1, weight_1_2)
        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(y_test[i:i + 1]))
    if j % 10 == 0:
        print(
            f'Iter {j} Test-acc: {test_correct_cnt / float(len(x_test))} Train-acc: {correct_cnt / float(len(x_train))}')

Iter 0 Test-acc: 0.4816 Train-acc: 0.2295
Iter 10 Test-acc: 0.7706 Train-acc: 0.7397450980392157
Iter 20 Test-acc: 0.8061 Train-acc: 0.7777941176470589
Iter 30 Test-acc: 0.8262 Train-acc: 0.8047156862745098
Iter 40 Test-acc: 0.8414 Train-acc: 0.8218529411764706
Iter 50 Test-acc: 0.8538 Train-acc: 0.833421568627451
Iter 60 Test-acc: 0.8607 Train-acc: 0.8429411764705882
Iter 70 Test-acc: 0.867 Train-acc: 0.8492647058823529
Iter 80 Test-acc: 0.8733 Train-acc: 0.8548431372549019
Iter 90 Test-acc: 0.878 Train-acc: 0.8593823529411765
Iter 100 Test-acc: 0.881 Train-acc: 0.8637941176470588
Iter 110 Test-acc: 0.8828 Train-acc: 0.8671274509803921
Iter 120 Test-acc: 0.886 Train-acc: 0.8703235294117647
Iter 130 Test-acc: 0.889 Train-acc: 0.8726274509803922
Iter 140 Test-acc: 0.8909 Train-acc: 0.8743921568627451


In [11]:
import json

weights_dict = {
    "weight_0_1": weight_0_1.T.tolist(),
    "weight_1_2": weight_1_2.T.tolist()
}

with open("./weights/weights.json", "w") as json_file:
    json.dump(weights_dict, json_file, indent=4)

print("Weights saved to weights.json")

Weights saved to weights.json


In [27]:
alpha, iterations = 2, 50
pixels, num_labels = 784, 10
batch_size = 200

In [28]:
inputs_rows, inputs_cols = 28, 28
kernel_rows, kernel_cols = 3, 3
num_kernels = 16

In [29]:
hidden_size = ((inputs_rows - kernel_rows) * (inputs_cols - kernel_cols)) * num_kernels

In [30]:
np.random.seed(1)
kernels = 0.02 * np.random.random((kernel_rows * kernel_cols, num_kernels)) - 0.01
weight_1_2 = 0.2 * np.random.random((hidden_size, num_labels)) - 0.1

In [31]:
def get_image_section(layer, row_from, row_to, col_from, col_to):
    section = layer[:, row_from:row_to, col_from:col_to]
    return section.reshape(-1, 1, row_to - row_from, col_to - col_from)

In [32]:
for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(x_train) / batch_size)):
        batch_start, batch_end = ((i * batch_size), (i * batch_size + batch_size))
        layer_0 = x_train[batch_start:batch_end]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0, row_start, row_start + kernel_rows, col_start,
                                         col_start + kernel_cols)
                sects.append(sect)
        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        kernel_output = np.dot(flattened_input, kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        dropout_mask = np.random.randint(2, size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1, weight_1_2))

        for k in range(batch_size):
            labelset = y_train[batch_start + k:batch_start + k + 1]
            _inc = int(np.argmax(layer_2[k:k + 1]) == np.argmax(labelset))
            correct_cnt += _inc

        layer_2_delta = (y_train[batch_start:batch_end] - layer_2) / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weight_1_2.T) * tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        weight_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        lid_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(lid_reshape)
        kernels -= alpha * k_update
    test_correct_cnt = 0
    for i in range(len(x_test)):
        layer_0 = x_test[i:i + 1]
        layer_0 = layer_0.reshape(layer_0.shape[0], 28, 28)
        layer_0.shape

        sects = list()
        for row_start in range(layer_0.shape[1] - kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0, row_start, row_start + kernel_rows, col_start,
                                         col_start + kernel_cols)
                sects.append(sect)
        expanded_input = np.concatenate(sects, axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0] * es[1], -1)
        kernel_output = np.dot(flattened_input, kernels)
        layer_1 = tanh(kernel_output.reshape(es[0], -1))
        layer_2 = softmax(np.dot(layer_1, weight_1_2))

        test_correct_cnt += int(np.argmax(layer_2) == np.argmax(y_test[i:i + 1]))
    if j % 10 == 0:
        print(
            f'Iter {j} Test-acc: {test_correct_cnt / float(len(x_test))} Train-acc: {correct_cnt / float(len(x_train))}')

Iter 0 Test-acc: 0.1541 Train-acc: 0.2502450980392157
Iter 10 Test-acc: 0.9247 Train-acc: 0.8787745098039216
Iter 20 Test-acc: 0.9249 Train-acc: 0.8885098039215686
Iter 30 Test-acc: 0.9219 Train-acc: 0.8920980392156863
Iter 40 Test-acc: 0.9123 Train-acc: 0.8641764705882353


In [46]:
import json

weights_dict = {
    "kernels": kernels.T.tolist(),
    "weight_1_2": weight_1_2.T.tolist()
}

with open("./weights/weightsCNN.json", "w") as json_file:
    json.dump(weights_dict, json_file, indent=4)

print("Weights saved to weights.json")

Weights saved to weights.json
